In [2]:
import scanpy as sc
import numpy as np
import tqdm
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from collections import Counter, defaultdict
import itertools as it
from utils import *

In [3]:
FILE = "datasets/10XGenomics/Targeted_SC3v3_Human_Glioblastoma_Neuroscience_filtered_feature_bc_matrix.h5"

In [22]:
adata = sc.read_10x_h5(FILE)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [29]:
Q = adata.X.todense()

AttributeError: 'numpy.ndarray' object has no attribute 'todense'

In [27]:
adata = ad.AnnData(reclassify(Q, 3))
X = adata.X
np.count_nonzero(X)

1850701

In [30]:
Q

matrix([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 5.,  0., 13., ...,  1.,  0.,  0.],
        ...,
        [ 0.,  0.,  2., ...,  0.,  0.,  0.],
        [ 1.,  0., 22., ...,  1.,  0.,  0.],
        [ 1.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [28]:
X

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 1., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [31]:
def cell(i):
    return np.array(X[i, :]).flatten()
ncells = X.shape[0]
sims = np.zeros((ncells, ncells))

cells = []
for i in tqdm.trange(ncells):
    cells.append(cell(i))


100%|███████████████████████████████████| 5697/5697 [00:00<00:00, 393093.09it/s]


In [34]:
for i in tqdm.trange(ncells):
    a = cells[i]
    q = a != 0
    for j in range(ncells):
        b = cells[j]
        sims[i][j] = sum(q & (a == b))

  0%|▏                                        | 19/5697 [00:04<24:28,  3.87it/s]


KeyboardInterrupt: 

In [39]:
i = 2
x = np.quantile(list(sims[i][:i]) + list(sims[i][i + 1:]), 0.9)
L = cells[i].shape[0]
out = np.full(L, True)
out &= cells[i] != 0

s_arr = list(zip(range(L), sims[i]))

q = 0
for k, v in sorted(s_arr, key = lambda i: -i[1]):
    if k == i or v < x:
        continue

    # print(k, v)
    out &= cells[i] == cells[k]
    print(sum(out))
    q += 1
print(q)

412
396
375
359
343
332
323
311
304
298
293
287
282
278
271
269
264
260
253
247
244
236
234
232
229
226
220
217
213
211
210
206
205
201
200
199
198
198
195
194
191
189
188
186
184
181
180
177
177
175
173
170
167
166
164
162
160
160
159
158
156
154
154
153
153
153
151
150
149
148
147
147
145
145
143
141
140
140
140
139
138
138
138
136
136
136
135
134
134
133
133
133
133
131
130
130
129
127
127
127
127
124
122
121
121
121
121
121
120
119
119
119
118
118
118
118
117
117
117
116
114
113
112
109
109
109
108
106
106
105
105
104
102
133


In [40]:
133 * 102

13566